### import libraries and load file

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.9 MB/s eta 0:00:00


In [2]:
import matplotlib.pyplot as plt
import numpy as np 
import os 
import pandas as pd 

In [21]:
nRowsRead = None # specify 'None' if want to read whole file
# labeled_data.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df0 = pd.read_csv('sample_data/FDCL18.csv', delimiter='\t', nrows = nRowsRead, skiprows=2, header=None)
df0.dataframeName = 'FDCL18.csv'
nRow, nCol = df0.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 99995 rows and 3 columns


In [26]:
df0.rename(columns={0 : 'text',
                    1 : 'category',
                    2 : 'majority'},inplace=True)
df0

,text,category,majority
0,RT @Papapishu: Man it would fucking rule if we...,abusive,4
1,It is time to draw close to Him &#128591;&#127...,normal,4
2,if you notice me start to act different or dis...,normal,5
3,"Forget unfollowers, I believe in growing. 7 ne...",normal,3
4,RT @Vitiligoprince: Hate Being sexually Frustr...,abusive,4
...,...,...,...
99990,RT @shangros: my fucking queen https://t.co/wa...,abusive,5
99991,#Osteporosis treated with #PEMF - rebuild bone...,normal,3
99992,@LGUSAMobile why does my phone screen keeps fl...,normal,3
99993,#bigdata vs. #reality ... but equally applies ...,normal,5


In [29]:
c

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
99990    NaN
99991    NaN
99992    NaN
99993    NaN
99994    NaN
Name: category, Length: 99995, dtype: object

In [31]:
## needs some refactoring...


a=df0['text']
b=df0['category']
df0.rename(columns={'category' : 'label'}, 
                    inplace=True)
c=df0['label'].map({'hateful':0, 'abusive':1,'spam':2, 'normal':3})

df= pd.concat([a,b,c], axis=1)
df.rename(columns={'majority' : 'label'}, 
                    inplace=True)
df

,text,category,label
0,RT @Papapishu: Man it would fucking rule if we...,abusive,1
1,It is time to draw close to Him &#128591;&#127...,normal,3
2,if you notice me start to act different or dis...,normal,3
3,"Forget unfollowers, I believe in growing. 7 ne...",normal,3
4,RT @Vitiligoprince: Hate Being sexually Frustr...,abusive,1
...,...,...,...
99990,RT @shangros: my fucking queen https://t.co/wa...,abusive,1
99991,#Osteporosis treated with #PEMF - rebuild bone...,normal,3
99992,@LGUSAMobile why does my phone screen keeps fl...,normal,3
99993,#bigdata vs. #reality ... but equally applies ...,normal,3


In [32]:
df.head()

,text,category,label
0,RT @Papapishu: Man it would fucking rule if we...,abusive,1
1,It is time to draw close to Him &#128591;&#127...,normal,3
2,if you notice me start to act different or dis...,normal,3
3,"Forget unfollowers, I believe in growing. 7 ne...",normal,3
4,RT @Vitiligoprince: Hate Being sexually Frustr...,abusive,1


In [33]:
df.groupby('label').count()

,text,category
label,,
0,4965,4965
1,27150,27150
2,14029,14029
3,53851,53851


# Split Dataset

In [34]:
import numpy as np
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_2, y_train, y_2 = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.2,
    random_state=17,
    stratify=df.label.values,    
)

In [36]:
X_val, X_test, y_val, y_test = train_test_split(
    X_2,
    y_2,
    test_size=0.5,
    random_state=17,
    stratify=y_2,    
)

In [37]:
# df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.loc[X_test, 'data_type'] = 'test'

In [38]:
df.groupby(['category', 'label', 'data_type']).count()

text
category label data_type       
abusive  1     test        2715
               train      21720
               val         2715
hateful  0     test         497
               train       3972
               val          496
normal   3     test        5385
               train      43081
               val         5385
spam     2     test        1403
               train      11223
               val         1403

# Tokenize

In [39]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import torch

In [40]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [41]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256
    # return_tensor='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    # return_tensor='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256
    # return_tensor='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [43]:
len(attention_masks_test[0])

256

In [44]:
dataset_train = TensorDataset(
    torch.tensor(input_ids_train),
    torch.tensor(attention_masks_train),
    labels_train
)

dataset_val = TensorDataset(
    torch.tensor(input_ids_val),
    torch.tensor(attention_masks_val),
    labels_val
)

dataset_test = TensorDataset(
    torch.tensor(input_ids_test),
    torch.tensor(attention_masks_test),
    labels_test
)

# Bert

In [45]:
from transformers import BertForSequenceClassification

In [42]:
label_dict = {'normal': 3, 'spam': 2, 'abusive': 1, 'hateful': 0}
label_dict

{'normal': 3, 'spam': 2, 'abusive': 1, 'hateful': 0}

In [ ]:
#possible_labels = df.category.unique()
#label_dict = {}
#for index, possible_label in enumerate(possible_labels):
#    label_dict[possible_label] = index
#label_dict = {'neither': 0, 'offensive_language': 1, 'hate_speech': 2}

In [46]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions=False,
    output_hidden_states=False    
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [47]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

### Create Dataloader

In [48]:
batch_size = 32

dataloader_train = DataLoader(
    dataset_train,
    # sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    # sampler=RandomSampler(dataset_val),
    batch_size=batch_size
)

dataloader_test = DataLoader(
    dataset_test,
    # sampler=RandomSampler(dataset_test),
    batch_size=batch_size
)


In [49]:
# Optimizer and Scheduler

from transformers import AdamW, get_linear_schedule_with_warmup

In [50]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)
epochs = 5
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [51]:
# Define Metrics
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [52]:
# Random seed for reproducibility
import random
from tqdm.notebook import tqdm

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [54]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

# Train

In [55]:
# epoch loop
#model.load_state_dict(torch.load('./finetuned_BERT2.model', map_location=torch.device('cpu')))
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

torch.save(model.state_dict(), f'finetuned_BERT.model')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2500 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.5143000535190105
Validation loss: 0.45722528125721806
F1 Score (Weighted): 0.8169728879403992


Epoch 2:   0%|          | 0/2500 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.43592268612980845
Validation loss: 0.4600459403885058
F1 Score (Weighted): 0.8195608910627825


...


Epoch 3
Training loss: 0.3935691419184208
Validation loss: 0.4757143018630366
F1 Score (Weighted): 0.8178451881147125


Epoch 4:   0%|          | 0/2500 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.3537764300972223
Validation loss: 0.5049251719785575
F1 Score (Weighted): 0.8076246702607496


Epoch 5:   0%|          | 0/2500 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.3265919584184885
Validation loss: 0.5241777090885388
F1 Score (Weighted): 0.8096828744333106


## Load Model & Evaluate

In [56]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

### Evaluation

In [57]:
model.load_state_dict(torch.load('finetuned_BERT.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [58]:
 test_loss, predictions, test_vals = evaluate(dataloader_test)
 accuracy_per_class(predictions, test_vals)

Class: hateful
Accuracy: 188/497

Class: abusive
Accuracy: 2510/2715

Class: spam
Accuracy: 749/1403

Class: normal
Accuracy: 4630/5385



In [59]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [60]:
test_loss, predictions, test_vals = evaluate(dataloader_test)
preds_flat = np.argmax(predictions, axis=1).flatten()
df.loc[df.data_type=='test','pred'] = preds_flat

train_loss, predictions, train_vals = evaluate(dataloader_train)
preds_flat = np.argmax(predictions, axis=1).flatten()
df.loc[df.data_type=='train','pred'] = preds_flat

val_loss, predictions, val_vals = evaluate(dataloader_val)
preds_flat = np.argmax(predictions, axis=1).flatten()
df.loc[df.data_type=='val','pred'] = preds_flat

In [61]:
df.to_csv('sample_data/1.csv')

In [ ]:
df.data_type=='test'

In [ ]:
test_vals.shape

    0->hate(1430)  1->Offensive(19190) 2->neither(4163)